In [136]:
seeds = ['1116667990060277760',\
 '1036736559939112960',\
 '2545545694',\
 '57108843',\
 '763393864229093376',\
 '34661052',\
 '1049876475917275137',\
 '29276200',\
 '728819442621153280']

In [127]:
def get_bot_activity(df,botlist,tot_tweets):
    """
    This function calculates the share of activity in a map on a given set of nodes
    Inputs:
        df: Subset of nodes dataframe based on given seedlist of nodes
        botlist: Given seedlist of nodes
        tot_tweets: Total number of tweets
    Outputs:
        bot_stats: A dictionary of calculcated stats on the given dataframe
    """
    bot_stats = {'bot_tweets':0,'total_tweets':tot_tweets}
    for bot in botlist:
        bot_stats['bot_tweets'] = bot_stats['bot_tweets'] + df[df.node_id == bot].shape[0]
    bot_stats['bot_share_percent'] = 100*(bot_stats['bot_tweets']/bot_stats['total_tweets'])
    for group in df.group_name.value_counts().index.tolist():
        bot_stats[group+'_count'] = df.group_name.value_counts()[group]
        bot_stats[group+'_share'] = 100*(bot_stats[group+'_count']/bot_stats['total_tweets'])
    return bot_stats

In [130]:
def run_bot_activity_report(debug = False):
    """
    This function runs the entirety of the Bot Activity Report (focused on a given seedlist of nodes)
    Inputs:
        debug: A Boolean to limit the amount of data pulled and processed for debugging purposes
    Outputs:
        result: The resulting Map Activity dataframe with both nodes and hits data
    """
    global debug_opt
    debug_opt = debug
    global map_id
    global map_features
    global map_feature_data
    
    input_map_id = int(input('>> Enter map ID: '))
    feature_type = input('>> What type of hit would you like to explore? (hashtags,retweets,urls)\n')
    if not debug_opt:
        seeds = list(input('>> Enter comma-separated seedlist: '))
    
    if input_map_id != map_id:
        print('...Fetching map data')
        map_id = input_map_id
        map_features = {'hashtags':pd.DataFrame(),'urls':pd.DataFrame(),'retweets':pd.DataFrame()}
    if map_features[feature_type].empty:
        print('...Fetching {} data'.format(feature_type))
        map_features[feature_type] = get_map_hits(feature_type,map_id)
        
        
#     hits_data = get_map_hits(hit_type,map_id)
#     display(hits_data)
#     return
    case_sensitive = input(">> Is this search case sensitive? (y/n) \n")
    if case_sensitive == 'n':
        case_sensitive = False
    else:
        case_sensitive = True
        
    hits_list = input('>> Enter a comma-separated list of hits you would like to examine: ')
    hits_list = hits_list.split(',')
    hits = {}
    for hit in hits_list:
        hit = hit.strip()
        hits_subset = map_features[feature_type][map_features[feature_type].hit_value == hit]
        hits_subset = map_features[feature_type][map_features[feature_type].hit_value.str.contains(hit,case=case_sensitive)]
        if not hits_subset.empty:
            hits[hit] = get_bot_activity(hits_subset,seeds,map_features[feature_type].shape[0])
        else:
            print('...{} is not present in map {}'.format(hit,map_id))
    result = pd.DataFrame(hits).transpose().fillna(0)
    display(result)
    if input('>> Do you want to save the bot activity results to a CSV? (y/n) \n') == 'y':
            print_csv(result)
    return